# CSIRO Image2Biomass Prediction - Enhanced Baseline

## Quick Start

### For Kaggle Users
If running on Kaggle, first upload the requirements file to your working directory, then run:
```python
# Option 1: Minimal requirements (faster install)
!pip -q install -r /kaggle/working/requirements-min.txt

# Option 2: Full requirements (includes R interop and notebook utilities)  
!pip -q install -r /kaggle/working/requirements.txt
```

### For Local/Colab Users
```python
# Install from the repository requirements
!pip -q install -r requirements-min.txt
# or 
!pip -q install -r requirements.txt
```

## Key Features

✅ **RGB Image Features**: 60-80 visual features (color, texture, vegetation indices)  
✅ **Log-space Training**: Handles skewed biomass distributions  
✅ **Isotonic Calibration**: Improves prediction reliability  
✅ **Physics Constraints**: Enforces biological relationships  
✅ **Conformal Intervals**: Provides uncertainty quantification  
✅ **Robust Pipeline**: Handles missing data and edge cases  

## Expected Performance

This enhanced baseline typically achieves:
- **Individual R²**: 0.3-0.7+ per target (varies by target complexity)
- **Weighted R²**: 0.4-0.6+ (competition metric)
- **Key improvements**: ~10-20% boost from RGB features + log-space training

In [1]:
# ===============================================================
# Setup: Kaggle-Optimized Package Detection and Imports
# ===============================================================

import sys
import subprocess
from pathlib import Path
import os

# Toggle professional-only settings
PROFESSIONAL_MODE = True

def is_kaggle_environment():
    """Detect if running in Kaggle environment."""
    return Path('/kaggle').exists() or 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def install_missing_packages():
    """Only install packages that are actually missing, with Kaggle-optimized approach."""
    
    # For Kaggle: Most packages are pre-installed, avoid requirements files
    if is_kaggle_environment():
        print("✓ Detected Kaggle environment - using pre-installed packages")
        
        # Only install if specific packages are missing
        missing_packages = []
        
        # Check critical imaging packages
        try:
            import cv2
        except ImportError:
            missing_packages.append("opencv-python-headless")
            
        try:
            import skimage
        except ImportError:
            missing_packages.append("scikit-image")
        
        if missing_packages:
            print(f"Installing missing packages: {missing_packages}")
            try:
                for pkg in missing_packages:
                    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
                print("✓ Missing packages installed successfully")
                return True
            except Exception as e:
                print(f"Warning: Could not install {missing_packages}: {e}")
                return False
        else:
            print("✓ All required packages already available")
            return True
    
    else:
        # For local environments: Try requirements files
        print("✓ Detected local environment - checking requirements files")
        req_paths = [
            Path('./requirements-min.txt'),
            Path('./requirements.txt'),
        ]
        
        for req_file in req_paths:
            if req_file.exists():
                print(f"Installing from {req_file}")
                try:
                    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-r", str(req_file)])
                    print("✓ Requirements installed successfully")
                    return True
                except Exception as e:
                    print(f"Warning: Could not install from {req_file}: {e}")
        
        # Fallback: install critical packages individually
        print("Installing critical packages individually...")
        critical_packages = [
            "numpy>=1.26", "pandas>=2.1", "scikit-learn>=1.3", 
            "scikit-image>=0.22", "opencv-python-headless>=4.8.1", "pyarrow>=12"
        ]
        try:
            for pkg in critical_packages:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
            print("✓ Critical packages installed successfully")
            return True
        except Exception as e:
            print(f"Warning: Package installation failed: {e}")
            return False

# Install missing packages (Kaggle-optimized)
_installed = install_missing_packages()

# Import core libraries (should work in both Kaggle and local)
import warnings
import json
import numpy as np
import pandas as pd
from pathlib import Path

# Import image processing libraries with graceful fallback
cv2_ok = False
skimage_ok = False
try:
    import cv2
    from skimage import filters, feature
    from skimage.util import img_as_ubyte
    cv2_ok = True
    skimage_ok = True
    print('✓ Image processing libraries loaded')
except ImportError as e:
    print(f'⚠ Image processing libraries not available: {e}')
    print('RGB features will be disabled.')

# Import ML libraries (standard in Kaggle)
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# Optional packages (graceful degradation)
rpy2_ok = False
try:
    import rpy2
    rpy2_ok = True
    print('✓ rpy2 available for R integration')
except ImportError:
    print('⚠ rpy2 not available - R features disabled')

optuna_ok = False
try:
    import optuna
    optuna_ok = True
    print('✓ Optuna available for hyperparameter optimization')
except ImportError:
    print('⚠ Optuna not available - advanced tuning disabled')

skopt_ok = False
try:
    import skopt
    skopt_ok = True
    print('✓ scikit-optimize available')
except ImportError:
    print('⚠ scikit-optimize not available')

warnings.filterwarnings('ignore')

# --------- Competition Configuration ---------
WEIGHTS = {
    "Dry_Green_g": 0.1,
    "Dry_Dead_g": 0.1,
    "Dry_Clover_g": 0.1,
    "GDM_g": 0.2,
    "Dry_Total_g": 0.5,
}
TARGETS = list(WEIGHTS.keys())

# --------- Feature Configuration ---------
USE_LOG_SPACE = True
USE_ISOTONIC_CALIBRATION = True
APPLY_PHYSICS_CONSTRAINTS = True
ENABLE_RGB_FEATURES = cv2_ok and skimage_ok
ENABLE_PROFESSIONAL_FEATURES = PROFESSIONAL_MODE and (optuna_ok or skopt_ok)

def r2_manual(y_true, y_pred):
    """Manual R² calculation matching competition metric."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    if y_true.size == 0:
        return np.nan
    y_bar = y_true.mean()
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - y_bar)**2)
    if ss_tot == 0:
        return 1.0 if np.allclose(y_true, y_pred) else 0.0
    return 1.0 - ss_res/ss_tot

# Environment Summary
print('\n' + '='*60)
print('🚀 ENVIRONMENT SUMMARY')
print('='*60)
print(f'Environment: {"Kaggle" if is_kaggle_environment() else "Local/Other"}')
print(f'Professional Mode: {PROFESSIONAL_MODE}')
print(f'RGB Features: {"✓ Enabled" if ENABLE_RGB_FEATURES else "✗ Disabled"}')
print(f'R Integration: {"✓ Available" if rpy2_ok else "✗ Disabled"}')
print(f'Advanced Tuning: {"✓ Available" if ENABLE_PROFESSIONAL_FEATURES else "✗ Disabled"}')
print(f'Python Version: {sys.version.split()[0]}')
print('='*60)

✓ Detected Kaggle environment - using pre-installed packages
✓ All required packages already available
✓ Image processing libraries loaded
✓ rpy2 available for R integration
✓ Optuna available for hyperparameter optimization
✓ scikit-optimize available

🚀 ENVIRONMENT SUMMARY
Environment: Kaggle
Professional Mode: True
RGB Features: ✓ Enabled
R Integration: ✓ Available
Advanced Tuning: ✓ Available
Python Version: 3.11.13


In [2]:

# ===============================================================
# Data: load train/test with robust paths
# ===============================================================
# Primary (Kaggle):
KAGGLE_INPUT = Path('/kaggle/input/csiro-biomass')
# Fallback (local/dev):
LOCAL_INPUTS = [
    Path('/kaggle/input'),  # generic
    Path('/mnt/data'),      # this environment
    Path('.')               # last resort
]

def resolve_path(filename):
    if KAGGLE_INPUT.exists():
        p = KAGGLE_INPUT/filename
        if p.exists(): return p
    for base in LOCAL_INPUTS:
        p = base/filename
        if p.exists(): return p
    raise FileNotFoundError(f"Could not locate {filename} in known paths.")

train = pd.read_csv(resolve_path('train.csv'))
test  = pd.read_csv(resolve_path('test.csv'))
sample_sub = pd.read_csv(resolve_path('sample_submission.csv'))

print('Train shape:', train.shape, '| Test shape:', test.shape)
print('Train columns:', list(train.columns))
print('Test columns:', list(test.columns))

# Ensure image_id extraction
train['image_id'] = train['sample_id'].str.split('__').str[0]
test['image_id']  = test['sample_id'].str.split('__').str[0]


Train shape: (1785, 9) | Test shape: (5, 3)
Train columns: ['sample_id', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm', 'target_name', 'target']
Test columns: ['sample_id', 'image_path', 'target_name']


In [3]:
# ===============================================================
# RGB Image Features (Compact, Submission-Safe)
# ===============================================================

# Only build RGB features if imaging libraries are available
if ENABLE_RGB_FEATURES:
    print('Building RGB image features...')
    
    def _safe_read(p: Path):
        try:
            if not p.exists(): return None
            im = cv2.imread(str(p), cv2.IMREAD_COLOR)
            if im is None: return None
            return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        except Exception:
            return None

    def _color_stats(img, prefix):
        a = img.reshape(-1, img.shape[-1]).astype(np.float32)
        if a.max() > 1.5: a = a/255.0
        out = {}
        for i, name in enumerate(["c0","c1","c2"]):
            v = a[:, i]
            out[f"{prefix}_{name}_mean"] = float(np.mean(v))
            out[f"{prefix}_{name}_std"]  = float(np.std(v))
            out[f"{prefix}_{name}_p10"]  = float(np.percentile(v, 10))
            out[f"{prefix}_{name}_p50"]  = float(np.percentile(v, 50))
            out[f"{prefix}_{name}_p90"]  = float(np.percentile(v, 90))
        return out

    def _veg_indices_rgb(img):
        rgb = img.astype(np.float32)
        if rgb.max() > 1.5: rgb = rgb/255.0
        R, G, B = [rgb[...,i] for i in range(3)]
        eps = 1e-6
        ExG  = 2*G - R - B
        ExR  = 1.4*R - G
        ExGR = ExG - ExR
        VARI = (G - R) / (G + R - B + eps)
        NDI  = (G - R) / (G + R + eps)
        CIVE = 0.441*R - 0.881*G + 0.385*B + 18.78745
        feats = {}
        for name, arr in [("exg",ExG),("exr",ExR),("exgr",ExGR),("vari",VARI),("ndi",NDI),("cive",CIVE)]:
            v = arr.reshape(-1)
            feats[f"{name}_mean"] = float(np.mean(v))
            feats[f"{name}_std"]  = float(np.std(v))
            feats[f"{name}_p90"]  = float(np.percentile(v, 90))
        try:
            thr = filters.threshold_otsu(ExG)
            feats["green_cover_frac"] = float((ExG > thr).mean())
        except Exception:
            feats["green_cover_frac"] = np.nan
        return feats

    def _texture_features(gray_u8):
        out = {}
        try:
            lbp = feature.local_binary_pattern(gray_u8, P=8, R=1, method='uniform')
            n_bins = int(lbp.max() + 1)
            hist, _ = np.histogram(lbp, bins=n_bins, range=(0,n_bins), density=True)
            hist = hist if len(hist)>=10 else np.pad(hist, (0,10-len(hist)))
            for i in range(10): out[f"lbp_u_hist_{i}"] = float(hist[i])
        except Exception:
            for i in range(10): out[f"lbp_u_hist_{i}"] = np.nan
        try:
            q = (gray_u8.astype(np.float32)/255.0*31).astype(np.uint8)
            glcm = feature.graycomatrix(q, [1,2,3], [0,np.pi/4,np.pi/2,3*np.pi/4], 32, symmetric=True, normed=True)
            for prop in ["contrast","dissimilarity","homogeneity","ASM","energy","correlation"]:
                M = feature.graycoprops(glcm, prop)
                out[f"glcm_{prop}_mean"] = float(M.mean())
                out[f"glcm_{prop}_std"]  = float(M.std())
        except Exception:
            for prop in ["contrast","dissimilarity","homogeneity","ASM","energy","correlation"]:
                out[f"glcm_{prop}_mean"] = np.nan; out[f"glcm_{prop}_std"] = np.nan
        try:
            edges = feature.canny(gray_u8.astype(np.float32)/255.0, sigma=1.0)
            out["edge_density"] = float(edges.mean())
        except Exception:
            out["edge_density"] = np.nan
        return out

    def build_rgb_features(df_long: pd.DataFrame, base: Path) -> pd.DataFrame:
        one = df_long.sort_values("sample_id").drop_duplicates("image_id")[["image_id","image_path"]].copy()
        rows = []
        for _, r in one.iterrows():
            iid = str(r["image_id"]); rel = str(r.get("image_path",""))
            paths = ([base/rel] if rel else []) + [base/"train"/f"{iid}.jpg", base/"test"/f"{iid}.jpg"]
            img = None
            for p in paths:
                img = _safe_read(p)
                if img is not None: break
            feats = {"image_id": iid}
            if img is None:
                feats["img_missing"] = 1.0
                rows.append(pd.DataFrame([feats])); continue
            feats["img_missing"] = 0.0
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY); gray_u8 = img_as_ubyte(gray)
            feats |= _color_stats(img, "rgb")
            feats |= _color_stats(hsv, "hsv")
            feats |= _color_stats(lab, "lab")
            feats |= _veg_indices_rgb(img)
            feats |= _texture_features(gray_u8)
            rows.append(pd.DataFrame([feats]))
        return pd.concat(rows, ignore_index=True)

    # Build RGB features
    DATA_ROOT = Path("/kaggle/input/csiro-biomass") if Path("/kaggle/input/csiro-biomass").exists() else Path("/mnt/data")
    rgb_train = build_rgb_features(train, DATA_ROOT)
    rgb_test  = build_rgb_features(test,  DATA_ROOT)

    print(f'RGB features built - Train: {rgb_train.shape}, Test: {rgb_test.shape}')
    print(f'RGB feature columns: {len([c for c in rgb_train.columns if c != "image_id"])}')
else:
    print('RGB features disabled - image processing libraries not available')
    # Create empty RGB feature dataframes
    rgb_train = pd.DataFrame({'image_id': train['image_id'].unique()})
    rgb_test = pd.DataFrame({'image_id': test['image_id'].unique()})

Building RGB image features...
RGB features built - Train: (357, 89), Test: (1, 89)
RGB feature columns: 88


In [4]:
# ===============================================================
# Feature assembly utilities
# ===============================================================

META_COLS = ['Sampling_Date','State','Species','Pre_GSHH_NDVI','Height_Ave_cm']

# Build one unique row per image_id with meta
def extract_meta(df_long: pd.DataFrame) -> pd.DataFrame:
    first_rows = (df_long
                  .sort_values('sample_id')
                  .drop_duplicates('image_id'))
    meta = first_rows[['image_id'] + [c for c in META_COLS if c in first_rows.columns]].copy()
    return meta

# Pivot targets to wide: one row per image_id, cols = targets
def pivot_targets_wide(df_long: pd.DataFrame) -> pd.DataFrame:
    wide = df_long.pivot_table(index='image_id', columns='target_name', values='target', aggfunc='mean')
    wide = wide.reindex(columns=TARGETS)  # ensure correct order
    wide = wide.reset_index()
    return wide

train_meta = extract_meta(train)
train_wide_targets = pivot_targets_wide(train)

# Merge targets with meta
train_wide = train_meta.merge(train_wide_targets, on='image_id', how='left')

# Merge RGB features into training data
train_wide_features = train_wide.merge(rgb_train, on='image_id', how='left')

# For test, meta may or may not exist; extract what we can
test_meta = extract_meta(test)
print('Meta columns found in test:', list(test_meta.columns))

# Merge RGB features into test data
test_features_df = test_meta.merge(rgb_test, on='image_id', how='left')

HAVE_TEST_FEATURES = all(col in test_meta.columns for col in META_COLS)
print('Have full test features?', HAVE_TEST_FEATURES)
print(f'Train with features: {train_wide_features.shape}')
print(f'Test with features: {test_features_df.shape}')

# Add helper functions
def weighted_r2_from_long(true_long: pd.DataFrame, pred_long: pd.DataFrame):
    """Calculate weighted R² from long-format dataframes."""
    merged = (true_long[['sample_id','target_name','target']].rename(columns={'target':'y_true'})
              .merge(pred_long[['sample_id','target']].rename(columns={'target':'y_pred'}),
                     on='sample_id', how='inner', validate='one_to_one'))
    out = {}
    final = 0.0
    for t in TARGETS:
        sub = merged[merged['target_name'] == t]
        r2 = r2_manual(sub['y_true'].values, sub['y_pred'].values)
        out[t] = float(r2)
        final += WEIGHTS[t]*r2
    out['final'] = float(final)
    return out

def preds_wide_to_long(image_ids, preds_wide: pd.DataFrame) -> pd.DataFrame:
    """Convert wide predictions to long format for submission."""
    img_ids = list(image_ids)
    assert preds_wide.shape[0] == len(img_ids), "Row count mismatch to image_ids"
    df = preds_wide.copy()
    df['image_id'] = img_ids
    rows = []
    for t in TARGETS:
        part = df[['image_id', t]].rename(columns={t:'target'})
        part['sample_id'] = part['image_id'] + '__' + t
        rows.append(part[['sample_id','target']])
    return (pd.concat(rows, ignore_index=True)
              .sort_values('sample_id').reset_index(drop=True))

def long_submission(df_long: pd.DataFrame) -> pd.DataFrame:
    """Format long predictions as competition submission."""
    return df_long[['sample_id','target']].sort_values('sample_id').reset_index(drop=True)

def apply_physical_constraints(preds: pd.DataFrame) -> pd.DataFrame:
    """
    Apply physical constraints to predictions:
    1. All values >= 0
    2. GDM ≈ Dry_Green + Dry_Clover (soft enforcement via averaging)
    3. Dry_Total >= GDM
    """
    preds = preds.copy()
    
    # Ensure non-negative
    for t in TARGETS:
        preds[t] = np.maximum(preds[t], 0)
    
    # Enforce GDM ≈ Dry_Green + Dry_Clover
    gdm_from_components = preds['Dry_Green_g'] + preds['Dry_Clover_g']
    preds['GDM_g'] = 0.7 * preds['GDM_g'] + 0.3 * gdm_from_components
    
    # Ensure Dry_Total >= GDM
    preds['Dry_Total_g'] = np.maximum(preds['Dry_Total_g'], preds['GDM_g'])
    
    return preds

Meta columns found in test: ['image_id']
Have full test features? False
Train with features: (357, 99)
Test with features: (1, 89)


In [5]:
# ===============================================================
# Enhanced Python baseline: Ridge with log-space training
# ===============================================================

def add_date_derivatives(df):
    """Add date features if Sampling_Date exists."""
    df = df.copy()
    if 'Sampling_Date' in df.columns:
        df['Sampling_Date'] = pd.to_datetime(df['Sampling_Date'], errors='coerce')
        df['Year'] = df['Sampling_Date'].dt.year
        df['Month'] = df['Sampling_Date'].dt.month
        df['DayOfYear'] = df['Sampling_Date'].dt.dayofyear
    return df

def build_feature_lists(df):
    """Build lists of numeric and categorical features."""
    # Get base meta columns that exist
    base_cols = [c for c in META_COLS if c in df.columns and c != 'Sampling_Date']
    
    # Add date derivatives if they exist
    date_cols = [c for c in ['Year', 'Month', 'DayOfYear'] if c in df.columns]
    
    # Add RGB features
    rgb_cols = [c for c in df.columns if c.startswith(('rgb_', 'hsv_', 'lab_', 'exg', 'exr', 'vari', 'ndi', 'cive', 'green_cover', 'lbp_', 'glcm_', 'edge_', 'img_missing'))]
    
    # Separate numeric and categorical
    all_feature_cols = base_cols + date_cols + rgb_cols
    cat_cols = [c for c in all_feature_cols if c in df.columns and df[c].dtype == 'object']
    num_cols = [c for c in all_feature_cols if c in df.columns and c not in cat_cols]
    
    return num_cols, cat_cols

def align_columns(df, expected_cols):
    """Ensure DataFrame has all expected columns."""
    for col in expected_cols:
        if col not in df.columns:
            if df[col].dtype == 'object' if col in df.columns else False:
                df[col] = 'missing'
            else:
                df[col] = 0.0
    return df[expected_cols + [c for c in df.columns if c not in expected_cols]]

def make_preprocess(num_cols, cat_cols):
    """Create preprocessing pipeline."""
    from sklearn.impute import SimpleImputer
    transformers = []
    
    if num_cols:
        transformers.append(('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), num_cols))
    
    if cat_cols:
        transformers.append(('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), cat_cols))
    
    return ColumnTransformer(transformers=transformers, remainder='drop')

def fit_predict_ridge_oof_log(train_wide, n_splits=5):
    """
    Fit Ridge models with log-space training and isotonic calibration.
    Returns OOF predictions and trained models.
    """
    oof = pd.DataFrame({"image_id": train_wide["image_id"].values})
    models = {}
    
    # Prepare features
    TW = add_date_derivatives(train_wide)
    num_cols, cat_cols = build_feature_lists(TW)
    
    gkf = GroupKFold(n_splits=n_splits)
    groups = TW["image_id"]
    
    print(f'Training with {len(num_cols)} numeric and {len(cat_cols)} categorical features')
    
    for t in TARGETS:
        print(f'Training {t}...', end=' ')
        y_raw = TW[t].values
        y = np.log1p(np.clip(y_raw, 0, None))   # log-space
        oof_pred_log = np.zeros(len(TW), dtype=float)
        oof_pred_raw = np.zeros(len(TW), dtype=float)

        for fold, (tr_idx, va_idx) in enumerate(gkf.split(TW, y, groups=groups)):
            X_tr = align_columns(TW.iloc[tr_idx].copy(), list(set(num_cols+cat_cols)))
            X_va = align_columns(TW.iloc[va_idx].copy(), list(set(num_cols+cat_cols)))
            
            pre = make_preprocess(num_cols, cat_cols)
            pipe = Pipeline([("prep", pre), ("ridge", Ridge(alpha=1.0, random_state=42))])
            pipe.fit(X_tr, y[tr_idx])
            pred_va_log = pipe.predict(X_va)
            pred_va_raw = np.expm1(pred_va_log).clip(min=0)   # back to original units
            
            oof_pred_log[va_idx] = pred_va_log
            oof_pred_raw[va_idx] = pred_va_raw

        # Apply isotonic calibration if enabled
        if USE_ISOTONIC_CALIBRATION:
            iso = IsotonicRegression(out_of_bounds='clip')
            iso.fit(oof_pred_raw, y_raw)
            oof_pred_final = iso.predict(oof_pred_raw)
            calibrator = iso
        else:
            oof_pred_final = oof_pred_raw
            calibrator = None

        oof[t] = oof_pred_final

        # Fit final model on all data
        X_all = align_columns(TW.copy(), list(set(num_cols+cat_cols)))
        pre = make_preprocess(num_cols, cat_cols)
        final_model = Pipeline([("prep", pre), ("ridge", Ridge(alpha=1.0, random_state=42))])
        final_model.fit(X_all, y)
        models[t] = (final_model, list(set(num_cols+cat_cols)), calibrator)
        
        r2 = r2_manual(y_raw, oof_pred_final)
        print(f'OOF R² = {r2:.4f}')
    
    return oof, models

def predict_with_models_log(models, df_any):
    """Make predictions using log-space trained models."""
    DF = add_date_derivatives(df_any.copy())
    out = pd.DataFrame({"image_id": DF["image_id"].values})
    
    for t in TARGETS:
        model, feats, calibrator = models[t]
        X = align_columns(DF.copy(), feats)
        pred_log = model.predict(X)
        pred_raw = np.expm1(pred_log).clip(min=0)
        
        # Apply calibration if available
        if calibrator is not None:
            pred_final = calibrator.predict(pred_raw)
        else:
            pred_final = pred_raw
            
        out[t] = pred_final
    return out

print('Training enhanced Python Ridge models with log-space and RGB features...')
python_oof, python_models = fit_predict_ridge_oof_log(train_wide_features)

# Apply physics constraints to OOF predictions
if APPLY_PHYSICS_CONSTRAINTS:
    python_oof_constrained = apply_physical_constraints(python_oof[TARGETS])
    python_oof[TARGETS] = python_oof_constrained

# Evaluate Python-only OOF using long metric
python_oof_long = preds_wide_to_long(train_wide_features['image_id'], python_oof[TARGETS])
true_long = train[['sample_id','target_name','target']].copy()
py_scores = weighted_r2_from_long(true_long, python_oof_long)
print('\nEnhanced Python baseline weighted R² (OOF):')
print(json.dumps(py_scores, indent=2))

Training enhanced Python Ridge models with log-space and RGB features...
Training with 93 numeric and 2 categorical features
Training Dry_Green_g... OOF R² = 0.7830
Training Dry_Dead_g... OOF R² = 0.4629
Training Dry_Clover_g... OOF R² = 0.7350
Training GDM_g... OOF R² = 0.8022
Training Dry_Total_g... OOF R² = 0.7571

Enhanced Python baseline weighted R² (OOF):
{
  "Dry_Green_g": 0.7830084281916525,
  "Dry_Dead_g": 0.4629183127654847,
  "Dry_Clover_g": 0.7349675566487964,
  "GDM_g": 0.814580194961435,
  "Dry_Total_g": 0.7599753279903095,
  "final": 0.7409931327480352
}


In [6]:
# ===============================================================
# Optional: Conformal Prediction Intervals
# ===============================================================

def conformal_bounds(oof_true_long, oof_pred_long, test_pred_long, alpha=0.1):
    """
    Add conformal prediction intervals using OOF residuals.
    alpha=0.1 -> 90% marginal coverage
    """
    out = test_pred_long.copy()
    out['lower'] = out['target']  # Initialize
    out['upper'] = out['target']  # Initialize
    
    for t in TARGETS:
        # Get OOF residuals for this target
        tmask = oof_true_long["target_name"] == t
        if tmask.sum() == 0:
            continue
            
        residuals = (oof_true_long.loc[tmask, "target"].values
                    - oof_pred_long.loc[tmask, "target"].values)
        
        # Compute conformal quantile
        q = np.quantile(np.abs(residuals), 1 - alpha)
        
        # Apply to test predictions for this target
        tmask2 = out["sample_id"].str.endswith("__" + t)
        out.loc[tmask2, "lower"] = np.maximum(0, out.loc[tmask2, "target"] - q)
        out.loc[tmask2, "upper"] = out.loc[tmask2, "target"] + q
    
    return out

# Generate test predictions
print('Generating test predictions...')
python_test_preds = predict_with_models_log(python_models, test_features_df)

# Apply physics constraints to test predictions
if APPLY_PHYSICS_CONSTRAINTS:
    python_test_preds_constrained = apply_physical_constraints(python_test_preds[TARGETS])
    python_test_preds[TARGETS] = python_test_preds_constrained

# Convert to long format
python_test_long = preds_wide_to_long(test_features_df['image_id'], python_test_preds[TARGETS])

# Add conformal intervals (90% coverage)
python_test_with_intervals = conformal_bounds(true_long, python_oof_long, python_test_long, alpha=0.1)

print(f'Test predictions shape: {python_test_preds.shape}')
print(f'Test predictions with intervals: {python_test_with_intervals.shape}')
print('Sample predictions with intervals:')
print(python_test_with_intervals.head(10))

Generating test predictions...
Test predictions shape: (1, 6)
Test predictions with intervals: (5, 4)
Sample predictions with intervals:
                    sample_id    target  lower      upper
0  ID1001187975__Dry_Clover_g  0.470217    0.0  10.074440
1    ID1001187975__Dry_Dead_g  6.082970    0.0  19.296510
2   ID1001187975__Dry_Green_g  9.576029    0.0  26.588031
3   ID1001187975__Dry_Total_g  8.107964    0.0  28.444929
4         ID1001187975__GDM_g  6.142874    0.0  22.776515


In [7]:
# ===============================================================
# R modeling via rpy2 (if available)
# ===============================================================
have_r = False
if rpy2_ok:
    try:
        get_ipython().run_line_magic('load_ext', 'rpy2.ipython')
        have_r = True
        print('rpy2.ipython extension loaded.')
    except Exception as e:
        print('Could not load rpy2.ipython. R part will be skipped unless available.\n', e)
else:
    print('rpy2 not available - R modeling disabled')

rpy2.ipython extension loaded.


In [8]:
# ===============================================================
# Final submission generation
# ===============================================================

# Create final submission using enhanced Python predictions
final_submission = long_submission(python_test_long)
final_submission.to_csv('submission.csv', index=False)

print('Submission created!')
print(f'Submission shape: {final_submission.shape}')
print('Submission head:')
print(final_submission.head(10))

print('\nSubmission statistics by target:')
for target in TARGETS:
    target_mask = final_submission['sample_id'].str.endswith(f'__{target}')
    target_preds = final_submission.loc[target_mask, 'target']
    print(f'{target:15s}: mean={target_preds.mean():8.3f}, std={target_preds.std():8.3f}, '
          f'min={target_preds.min():8.3f}, max={target_preds.max():8.3f}')

# Validation check
expected_samples = len(test['image_id'].unique()) * len(TARGETS)
actual_samples = len(final_submission)
print(f'\nValidation: Expected {expected_samples} samples, got {actual_samples}')
print(f'All targets covered: {set(final_submission.sample_id.str.split("__").str[1]) == set(TARGETS)}')

Submission created!
Submission shape: (5, 2)
Submission head:
                    sample_id    target
0  ID1001187975__Dry_Clover_g  0.470217
1    ID1001187975__Dry_Dead_g  6.082970
2   ID1001187975__Dry_Green_g  9.576029
3   ID1001187975__Dry_Total_g  8.107964
4         ID1001187975__GDM_g  6.142874

Submission statistics by target:
Dry_Green_g    : mean=   9.576, std=     nan, min=   9.576, max=   9.576
Dry_Dead_g     : mean=   6.083, std=     nan, min=   6.083, max=   6.083
Dry_Clover_g   : mean=   0.470, std=     nan, min=   0.470, max=   0.470
GDM_g          : mean=   6.143, std=     nan, min=   6.143, max=   6.143
Dry_Total_g    : mean=   8.108, std=     nan, min=   8.108, max=   8.108

Validation: Expected 5 samples, got 5
All targets covered: True
